In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("openai-community/roberta-large-openai-detector")
model = AutoModelForSequenceClassification.from_pretrained("openai-community/roberta-large-openai-detector")

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F
from pathlib import Path
import json
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [6]:
text = "This is a sample text that may have been written by AI."

inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
inputs = {k: v.to(device) for k, v in inputs.items()}

# Get model predictions
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    probs = F.softmax(logits, dim=-1)

# Print probabilities and predicted class
labels = ["human", "AI"]  # According to the model card
predicted_label = labels[probs.argmax().item()]
print("Predicted:", predicted_label)
print("Probabilities:", dict(zip(labels, probs.squeeze().tolist())))

Predicted: AI
Probabilities: {'human': 0.43415388464927673, 'AI': 0.5658460855484009}


In [ ]:
test_df = pd.read_csv('../Splits/combined/sub_test.csv')
# test_df = pd.read_csv('../Splits/harder_test/sub_test.csv')
test_df

In [7]:
def GenerateResult(source: str):
  inputs = tokenizer(source, return_tensors="pt", truncation=True, padding=True)
  inputs = {k: v.to(device) for k, v in inputs.items()}
  with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    probs = F.softmax(logits, dim=-1)
  labels = ["human", "AI"]  # According to the model card
  predicted_label = labels[probs.argmax().item()]
  return dict(zip(labels, probs.squeeze().tolist()))

In [11]:
GenerateResult(test_df['source'][0])

{'human': 0.4041837751865387, 'AI': 0.5958161950111389}

In [12]:
import pathlib
import httpx
import time
from IPython.display import clear_output
err_logs = []
answer = []

err_cnt = 0

for index, row in test_df[len(answer):].iterrows():
  while True:
    try:
      response = GenerateResult(row['source'])
      clear_output()
      print(f"ASKED : {index}")
      err_cnt = 0
      answer.append(response)
      break
    except Exception as e:
      err_cnt += 1
      err_logs.append((index,e))
      clear_output()
      print(response)
      print(f"ERROR at {index} : {e}")

ASKED : 2999


In [ ]:
import re
import json


result_n = len(answer)
answer_df = test_df.iloc[:result_n]
answer_df['answer_dict'] = answer

In [ ]:
answer_df['AI_percentage'] = answer_df['answer_dict'].apply(lambda x: x['AI'])
answer_df['Human_percentage'] = answer_df['answer_dict'].apply(lambda x: x['human'])

In [ ]:
answer_df['Model_Answer'] = answer_df['AI_percentage'].apply(
    lambda x: 'AI' if x >= 0.5 else 'human'
)

In [ ]:
answer_df['verdict'] = answer_df['writer'] == answer_df['Model_Answer']
answer_df = answer_df[['writer','Model_Answer','verdict','AI_percentage','website','source']]

In [23]:
filtered_df = pd.DataFrame()
filtered_df[['Label','Model_Answer','Verdict','AI-Percentage','Website']] = answer_df[['writer','Model_Answer','verdict','AI_percentage','website']]

In [24]:
filtered_df['Model_Answer'] = filtered_df['Model_Answer'].replace('Human', 'human')
filtered_df['Label'] = filtered_df['Label'].replace('Human', 'human')
filtered_df['Verdict'] = filtered_df['Label']==filtered_df['Model_Answer']
filtered_df = filtered_df[filtered_df['Model_Answer'].isin(['human', 'AI'])]
filtered_df

,Label,Model_Answer,Verdict,AI-Percentage,Website
0,AI,AI,True,0.595816,https://codeforces.com
1,AI,AI,True,0.511644,https://codeforces.com
2,AI,AI,True,0.527617,https://programming.in.th
3,human,AI,False,0.845653,https://codeforces.com
4,AI,human,False,0.474799,https://programming.in.th
...,...,...,...,...,...
2995,human,human,True,0.409265,https://programming.in.th
2996,human,AI,False,0.582648,https://programming.in.th
2997,human,AI,False,0.529819,https://firefly.gchan.moe
2998,AI,AI,True,0.631741,https://codeforces.com


In [ ]:
filtered_df.to_csv("./Results/_2/Result-RoBERTaLargeOpenAI-Detector.csv",index=False)

In [25]:
accuracy = filtered_df['Verdict'].value_counts(normalize=True).get(True, 0)
print(f"AUC : {accuracy}")

AUC : 0.461
